In [22]:
import cv2
import numpy as np

In [2]:
def histogram_block_matching(target_img, template_hist, block_size, stride, threshold):
    target_gray = cv2.cvtColor(target_img, cv2.COLOR_BGR2GRAY)
    target_height, target_width = target_gray.shape
    potential_locations = []
    for y in range(0, target_height - block_size[0] + 1, stride):
        for x in range(0, target_width - block_size[1] + 1, stride):
            block = target_gray[y:y+block_size[0], x:x+block_size[1]]
            
            block_hist = cv2.calcHist([block], [0], None, [256], [0, 256])
            block_hist = cv2.normalize(block_hist, block_hist, 0, 1, cv2.NORM_MINMAX, -1)
            
            match_score = cv2.compareHist(block_hist, template_hist, cv2.HISTCMP_BHATTACHARYYA)
            if match_score < threshold:
                potential_locations.append((x, y))
    
    return potential_locations

In [23]:
target_img = cv2.imread("current_frame.jpg")
template_img = cv2.imread("car_template.jpg")

In [24]:
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)

In [25]:
template_hist = cv2.calcHist([template_gray], [0], None, [256], [0, 256])
template_hist = cv2.normalize(template_hist, template_hist, 0, 1, cv2.NORM_MINMAX, -1)


In [27]:
block_size = (template_img.shape[0], template_img.shape[1])  # Use template size as block size
stride = 50  # Adjust as needed


In [28]:
threshold = 0.5

In [29]:
potential_locations =[]

In [30]:
potential_locations = histogram_block_matching(target_img, template_hist, block_size, stride, threshold)

In [32]:
for loc in potential_locations:
    x, y = loc
    cv2.rectangle(target_img, (x, y), (x + block_size[1], y + block_size[0]), (0, 255, 0), 2)


In [34]:
# Display result
cv2.imshow("Object Detection", target_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Potential to improve: Apply NMS thresholding to supress the bounding boxes with lower probabilities

In [38]:
def covariance_object_detection(target_img, template_covariance, window_size):
    # Convert target image to grayscale
    target_gray = cv2.cvtColor(target_img, cv2.COLOR_BGR2GRAY)
    
    # Calculate the covariance matrix of image patches using sliding window
    rows, cols = target_gray.shape
    detection_map = np.zeros((rows - window_size[0] + 1, cols - window_size[1] + 1))

    for i in range(rows - window_size[0] + 1):
        for j in range(cols - window_size[1] + 1):
            window = target_gray[i:i+window_size[0], j:j+window_size[1]]
            window_covariance = np.cov(window)
            distance = np.linalg.norm(np.subtract(window_covariance, template_covariance))
            detection_map[i, j] = distance
    
    return detection_map

In [39]:
template_covariance = np.cov(template_gray)

In [40]:
window_size = (template_img.shape[0], template_img.shape[1]) 

In [41]:
detection_map = covariance_object_detection(target_img, template_covariance, window_size)

In [42]:
detection_map = (detection_map - np.min(detection_map)) / (np.max(detection_map) - np.min(detection_map))

In [17]:
detection_map

array([[0.48603406, 0.4865614 , 0.48718375, ..., 0.44369342, 0.43942709,
        0.43576579],
       [0.48724805, 0.48777529, 0.48847326, ..., 0.4208755 , 0.41790148,
        0.41608362],
       [0.48666385, 0.48690614, 0.48757949, ..., 0.42126419, 0.41751433,
        0.4150783 ],
       ...,
       [0.45938729, 0.4592682 , 0.45960282, ..., 0.56425155, 0.55825641,
        0.55404204],
       [0.46064577, 0.46052971, 0.46072158, ..., 0.57206962, 0.56638107,
        0.56215495],
       [0.46370382, 0.46378057, 0.46408007, ..., 0.57727978, 0.57187364,
        0.56754839]])

In [43]:
threshold = 0.8# Adjust threshold as needed
potential_locations = np.where(detection_map > threshold)

In [44]:
potential_locations 

(array([ 28,  28,  28, ..., 187, 187, 187]),
 array([117, 118, 119, ..., 239, 240, 241]))

In [45]:
# Visualize potential object locations
for i, j in zip(*potential_locations):
    cv2.rectangle(target_img, (j, i), (j + window_size[1], i + window_size[0]), (0, 255, 0), 2)


In [46]:
cv2.imshow("Object Detection", target_img)
cv2.waitKey(0)
cv2.destroyAllWindows()